<a href="https://colab.research.google.com/github/Sylvariane/Analyse-vente/blob/master/P04_01_scriptdonn%C3%A9es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script de nettoyage des données

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Téléchargements des modules et des fichiers

Pour le nettoyage des données, on utilise trois librairies dont deux spécifiques aux Data Sciences : Pandas et NumPy. On importe aussi la librairie datetime pour modifier les dates en un objet spécifique 'Date'.

In [5]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

Une fois les différents modules importés, on charge les fichiers bruts nécessaires à notre analyse. On dispose d'un jeu de données composé de trois fichiers : un fichier contenant les ventes, nommé transactions, mais qui sera renommé sells ; un fichier contenant les informations concernant les clients nommé customers puis renommé custom et un fichier contenant les informations les produits vendus se nommant products et qui sera renommé prod pour le reste de l'analyse.

In [6]:
sells = pd.read_csv('/content/drive/My Drive/Formation Data Analyst/Livrables/P4_guillot_cecile/sources/transactions.csv')
custom = pd.read_csv('/content/drive/My Drive/Formation Data Analyst/Livrables/P4_guillot_cecile/sources/customers.csv')
prod = pd.read_csv('/content/drive/My Drive/Formation Data Analyst/Livrables/P4_guillot_cecile/sources/products.csv')

FileNotFoundError: ignored

Dans un premier temps, les fichiers seront nettoyés un par un puis ensuite, ils seront croisés. Dans une dernière partie, les fichiers crées seront enregistrés pour être analysé. 

## 2. Nettoyage des données du fichier des transactions

On commence par afficher les premières lignes du DataFrame pour voir sa composition. 

In [ ]:
print(sells.head())

Le DataFrame des ventes contient quatre colonnes : l'identifiant des produits, la date de la transaction, le numéro de la session et l'identifiant du client acheteur.

### 2.1. Analyse des données manquantes

On affiche ensuite les données manquantes pouvant s'être introduite dans le fichier. Elles sont notées par le symbole NaN.

In [ ]:
sells.isna().any()

Le fichier contenant les informations sur les transactions ne présentent pas de données manquantes. 

### 2.2. Analyse des données dupliquées

On crée une variable qui contient les données dupliquées. Ensuite, on affiche les données dupliquées du DataFrame. 

In [ ]:
duplicated_sells = sells.duplicated()
sells[duplicated_sells]

On observe la présence de 126 lignes correspondant à des sessions de tests. Il faudra donc les supprimer du fichier des transactions mais aussi des autres fichiers. Il faudra enlever le produit ayant un id_prod correspond à T_0 dans le fichier des produits et les clients dont le client_id correspond à ct_0 et ct_1. 

On supprime donc les valeurs dupliquées correspond aux sessions test du 1er mars.

In [ ]:
pd.set_option('mode.chained_assignment', None)
sells = sells.drop_duplicates()
sells[duplicated_sells]

Les valeurs dupliquées sont donc supprimées mais il reste encore des données de session test avec l'identifiant s_0. On va donc supprimer ces valeurs pour pouvoir faire son analyse. 

In [ ]:
sells.loc[sells['session_id'] == 's_0']

Pour supprimer toutes les données issues des tests, on supprime toutes les sessions dont l'identifiant session vaut 's_0'. On crée donc une liste contenant les index des lignes avec l'identifiant de session s_0 puis on supprime les lignes associées à l'aide de la méthode .drop().

In [ ]:
index_s_0 = sells[sells['session_id'] == 's_0'].index.values
sells.drop(index_s_0, 0, inplace=True)

In [ ]:
sells.loc[sells['session_id'] == 's_0']

### 2.3. Analyse des types

Une fois ces différents nettoyages effectués, on s'intéresse aux types de données contenus dans le fichier des transactions.

In [ ]:
print(sells.dtypes)

On remarque que toutes les variables sont considérées comme des objets. On peut transformer la variable 'date' en type 'datetime'.

In [ ]:
sells['date'] = pd.to_datetime(sells['date'])
print(sells.dtypes)

Une fois cette transformation faite, il est plus simple d'organiser les données par ordre chronologique. 

In [ ]:
sells = sells.sort_values('date')
sells = sells.set_index('date')
print(sells.head())

## 3. Nettoyage des données des produits

Comme pour le fichier des transactions, on affiche les premiers lignes du fichier des produits pour pouvoir se familiariser avec ce fichier et voir sa composition.

In [ ]:
print(prod.head())

Ce dataframe est composé de trois colonnes contenant les identifiants des produits, les prix des produits et la catégorie d'appartenance de chacun des produits.

### 3.1. Analyse des types dans le fichier des produits

On regarde le détail des types de données contenues dans le dataframe.

In [ ]:
prod.dtypes

Il serait plus intéressant de modifier le type de la colonne 'categ' en une valeur catégorielle. 

In [ ]:
prod['categ'] = prod['categ'].astype('category')
prod.dtypes

On peut ainsi classer les produits par ordre de catégorie. 

In [ ]:
prod = prod.sort_values('categ')
print(prod)

### 3.2. Analyse des valeurs manquantes

Après la modification des types, on s'intéresse à la présence de données manquantes qui vont prendre la forme de NaN dans notre DataFrame. Pour identifier ces valeurs, on utiliser la fonction isna() associée à any().

In [ ]:
prod.isna().any()

Le DataFrame des produits ne contient aucune donnée manquante. 

### 3.3. Analyse des valeurs dupliquées

Ensuite, on s'intéresse au valeur dupliquée. On crée une variable qui va contenir les valeurs dupliquées dans notre table pour pouvoir les isoler et voir la forme de ces valeurs dupliquées.

In [ ]:
duplicated = prod.duplicated()
prod[duplicated]

Le DataFrame prod ne présente pas de valeur dupliqué. On peut donc le laisser ainsi. 

### 3.4. Suppression du produit T_0

En effectuant le nettoyage du fichier des transactions, on a observé la présence d'un produit dont la valeur était T_0. Sachant que les données associées aux sessions test ont été supprimé dans le fichier des transactions, il peut être intéressant de supprimer ce produit T_0 pour ne pas fausser les analyses qui suivront. 

In [ ]:
t_0 = prod.loc[prod['id_prod'] == 'T_0']
print(t_0)

Maintenant que l'on a identifié ce produit T_0, on va pouvoir le supprimer du fichier en utilisant la méthode .drop.

In [ ]:
prod.drop(731, 0, inplace=True)

On vérifie que la donnée a été supprimé du fichier des produits. 

In [ ]:
t_0 = prod.loc[prod['id_prod'] == 'T_0']
print(t_0)

## 4. Nettoyage des données des clients

Concernant le fichier client, la méthode sera la même que pour les produits. On commence d'abord par afficher les premières lignes du DataFrame ainsi que les types pour voir la composition de celui-ci. 

In [ ]:
print(custom.head())

Ce Dataframe est lui aussi composé de trois colonnes contenant les identifiants des clients, le sexe et l'année de naissance des clients. 

### 4.1. Analyse des types

On s'intéresse ensuite aux types des données présents dans le fichier client.

In [ ]:
print(custom.dtypes)

La variable 'sex' apparaît comme un objet, il peut être plus intéressant de changer ce type en variable catégorielle. 

In [ ]:
custom['sex'] = custom['sex'].astype('category')
custom.dtypes

### 4.2. Analyse des données manquantes

On cherche la présence de valeurs manquantes (NaN) dans le fichier client. 

In [ ]:
custom.isna().any()

Le fichier client ne présente pas de données manquantes.

### 4.3. Analyse des données dupliquées

On regarde la présence de données dupliquées et on les stocke dans une variable que l'on nomme duplicated_custom.

In [ ]:
duplicated_custom = custom.duplicated()
custom[duplicated_custom]

Ce DataFrame ne contient pas de données dupliquées. 

### 4.4. Suppresion des clients ct_0 et ct_1

Lors du nettoyage du fichier des transactions, on avait vu apparaître des clients associés à des sessions de test. Ces clients possédaient les identifiants ct_0 et ct_1. On va donc supprimer ces deux clients pour ne pas fausser les analyses qui suivront.

In [ ]:
ct_0 = custom.loc[custom['client_id'] == 'ct_0']
ct_1 = custom.loc[custom['client_id'] == 'ct_1']
print(ct_0)
print(ct_1)

On supprime ces deux valeurs en utilisant la méthode .drop(). 

In [ ]:
custom.drop([2735, 8494], 0, inplace=True)
custom.sort_values('client_id')

## 5. Analyse et croisement avec les autres DataFrames

On cherche ensuite si des id_prod sont présents dans les ventes mais pas dans le fichier produit. Si c'est le cas, on supprime ce produit car il ne possède pas de prix. On fait de même avec les client_id. POur cela, on crée deux colonnes contenant des booléans permettant de vérifier la présence de ces anomalies.

In [ ]:
sells['id_prod_prod'] = sells['id_prod'].isin(prod['id_prod'])
sells['client_id_custom'] = sells['client_id'].isin(custom['client_id'])
print(sells.head())

On affiche les produits pour lesquels la variable id_prod_prod vaut False. Cela signifie que ce code produit se trouve uniquement dans les ventes mais pas dans les produits. 

In [ ]:
id_prod_false = sells[sells['id_prod_prod'] == False]
id_prod_false = id_prod_false.groupby('id_prod').mean()
print(id_prod_false)

On remarque que cela concerne uniquement un produit avec l'identifiant '0_2245'.  

In [ ]:
prod_0_2245 = prod.loc[prod['id_prod'] == '0_2245']
print(prod_0_2245)

Après vérification dans le DataFrame des produits, il n'existe pas de produit avec cet identifiant. On peut donc essayer d'imputer la valeur de ce produit en lui donnant la moyenne des prix de sa catégorie. 

On répète les étapes précédentes pour repérer la présence de client existants dans les sessions de vente mais qui ne sont pas existants dans le fichier.

In [ ]:
id_client_false = sells[sells['client_id_custom'] == False]
id_client_false = id_client_false.groupby('client_id_custom').mean()
print(id_client_false)

Il n'y a pas de valeur 'False', on ne supprime pas de ligne pour cette condition. 

On calcule donc la moyenne des prix des produits de catégorie. 

In [ ]:
transactions = pd.merge(sells, prod, on=['id_prod']) 
transactions = pd.pivot_table(index='id_prod', columns='categ', values='price', aggfunc=np.mean, data=transactions)
moy_cat0 = transactions[0].mean(skipna=True)
print('Prix moyen dans la catégorie 0:', moy_cat0)

On ajoute donc une nouvelle ligne au fichier des produits pour y ajouter le produit 0_2245 avec un prix de 11.71.  

In [ ]:
prod = prod.sort_index()
print(prod.tail())

In [ ]:
prod.loc[3287] = {'id_prod' : '0_2245', 'price' : 11.71, 'categ' : 0}
print(prod.tail())

Les ventes associées à ce produit ont bien été ajoutés à ce fichier. On va pouvoir l'importer de cette manière pour pouvoir faire l'analyse des ventes.

In [ ]:
sells = sells.drop(columns=['client_id_custom', 'id_prod_prod'])

In [ ]:
df = pd.merge(sells, custom, on='client_id')
df = pd.merge(df, prod, on = 'id_prod')
df['count'] = 1
df = df.groupby('client_id').sum().reset_index()
df = df.sort_values('count', ascending=False)
df = df_custom[['client_id', 'count']]
df = pd.merge(df, custom, on='client_id')
df = df.sort_values('count', ascending=False)
top_10 = df.iloc[0:10]
print(top_10)

In [ ]:
mask = df.loc[(df['client_id'] == 'c_1609') | (df['client_id'] =='c_6714') | (df['client_id'] =='c_3454') | (df['client_id'] =='c_4958')]
outliers = mask.index.tolist()
sells = sells.drop(outliers)

Une fois les corrections faites, on enregistre le fichier pour l'étape de l'analyse. 

## 6. Enregistrements des fichiers nettoyés

Une fois le nettoyage des fichiers terminés, on les enregistre pour pouvoir avoir accès aux fichiers d'origine si on remarque un problème lors de l'analyse. Les fichiers source sont conservés dans un dossier 'source' qui se retrouven dans le chemin d'appel du début de ce script. 

In [ ]:
sells.to_csv('sells.csv')

In [ ]:
prod.to_csv('prod.csv')

In [ ]:
custom.to_csv('custom.csv')

## 7. Analyses complémentaires concernant le mois d'octobre

Après une analyse du chiffre d'affaire, on observe une anomalie pour le mois d'octobre. On va donc se pencher sur les ventes du mois d'octobre pour pouvoir expliquer cette anomalie. 

In [ ]:
sells = sells.reset_index()
sells_2 = pd.merge(sells, prod, on = ['id_prod'])
sells_2.head()

In [ ]:
sells_2['mois-année'] = pd.to_datetime(sells_2['date'])
sells_2['mois-année'] = sells_2['date'].apply(lambda x: x.strftime("%b %Y"))
sells_2.head()

In [ ]:
sells_sept_oct_nov = sells_2[(sells_2['mois-année'] == 'Sep 2021') |(sells_2['mois-année'] == 'Oct 2021') | (sells_2['mois-année'] == 'Nov 2021')]

sns.set_style('ticks')
sns.set_context('talk')
sns.set_palette('Set1')

_ = sns.catplot('mois-année', data=sells_sept_oct_nov, hue='categ', kind='count', height=10)
_ = plt.xticks(rotation=45)
_ = plt.xlabel('Categorie')
_ = plt.ylabel('Nombre de ventes')
_ = plt.title('Nombre de ventes en fonction du mois')

_ = plt.show()

In [ ]:
pd.set_option('mode.chained_assignment', None)
sells_oct = sells_2[sells_2['mois-année'] == 'Oct 2021']
sells_oct['date'] = sells_oct['date'].apply(lambda x: x.strftime("%a %d %b %Y"))
sells_oct['date'] = pd.to_datetime(sells_oct['date'])
sells_oct = sells_oct.sort_values('date')
sells_oct['date'] = sells_oct['date'].apply(lambda x: x.strftime("%a %d %b %Y"))
print(sells_oct)

In [ ]:
sns.set_style('ticks')
sns.set_context('talk')
sns.set_palette('Set1')

_ = sns.catplot('date', data=sells_oct, hue='categ', kind='count', height=10)
_ = plt.xticks(rotation=90)
_ = plt.xlabel('Date')
_ = plt.ylabel('Nombre de ventes')
_ = plt.title('Nombre de ventes en fonction de la catégorie pour le mois d\'octobre 2021')

_ = plt.show()